In [17]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import os
from urllib.request import urlopen,urlretrieve
from PIL import Image
from tqdm import tqdm_notebook
%matplotlib inline
from sklearn.utils import shuffle
import cv2
from resnets_utils import *

from keras.models import load_model
from sklearn.datasets import load_files   
from keras.utils import np_utils
from glob import glob
from keras import applications
from keras.preprocessing.image import ImageDataGenerator 
from keras import optimizers
from keras.models import Sequential,Model,load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D,GlobalAveragePooling2D
from keras.callbacks import TensorBoard,ReduceLROnPlateau,ModelCheckpoint

In [18]:
X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = load_dataset()

# Normalize image vectors
X_train = X_train_orig/255.
X_test = X_test_orig/255.

# Convert training and test labels to one hot matrices
Y_train = convert_to_one_hot(Y_train_orig, 6).T
Y_test = convert_to_one_hot(Y_test_orig, 6).T

print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

number of training examples = 1080
number of test examples = 120
X_train shape: (1080, 64, 64, 3)
Y_train shape: (1080, 6)
X_test shape: (120, 64, 64, 3)
Y_test shape: (120, 6)


In [20]:
img_height,img_width = 64,64 
num_classes = 6
#If imagenet weights are being loaded, 
#input must have a static square shape (one of (128, 128), (160, 160), (192, 192), or (224, 224))
base_model = applications.resnet50.ResNet50(weights= None, include_top=False, input_shape= (img_height,img_width,3))

C:\ProgramData\Anaconda3\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [21]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.7)(x)
predictions = Dense(num_classes, activation= 'softmax')(x)
model = Model(inputs = base_model.input, outputs = predictions)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [22]:
from keras.optimizers import SGD, Adam
# sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
adam = Adam(lr=0.0001)
model.compile(optimizer= adam, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(X_train, Y_train, epochs = 100, batch_size = 64)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/100
1080/1080 [==============================] - ETA: 3:41 - loss: 3.9475 - acc: 0.156 - ETA: 2:18 - loss: 3.9300 - acc: 0.179 - ETA: 1:48 - loss: 3.9126 - acc: 0.182 - ETA: 1:30 - loss: 3.9817 - acc: 0.168 - ETA: 1:19 - loss: 3.8388 - acc: 0.190 - ETA: 1:10 - loss: 3.7395 - acc: 0.197 - ETA: 1:01 - loss: 3.7802 - acc: 0.183 - ETA: 53s - loss: 3.7264 - acc: 0.191 - ETA: 46s - loss: 3.6914 - acc: 0.18 - ETA: 39s - loss: 3.6899 - acc: 0.18 - ETA: 33s - loss: 3.7142 - acc: 0.18 - ETA: 27s - loss: 3.6575 - acc: 0.18 - ETA: 21s - loss: 3.6596 - acc: 0.18 - ETA: 15s - loss: 3.6555 - acc: 0.19 - ETA: 10s - loss: 3.5877 - acc: 0.19 - ETA: 4s - loss: 3.5426 - acc: 0.2012 - 91s 84ms/step - loss: 3.5248 - acc: 0.1981
Epoch 2/100
1080/1080 [==============================] - ETA: 1:16 - loss: 2.8374 - acc: 0.156 - ETA: 1:13 - loss: 2.7317 - acc: 0.242 - ETA: 1:08 - loss: 2.7179 - acc: 0.276 - ETA: 

In [ ]:
preds = model.evaluate(X_test, Y_test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

In [ ]:
model.summary()